In [11]:
import itertools
import string
import random
import json
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)

In [2]:
def generate_row_letters(num_strings: int, max_length: int = 2):
    counter = 0
    for length in range(1, max_length + 1):
        if counter >= num_strings:
            break
        for item in itertools.product(string.ascii_uppercase, repeat=length):
            if counter >= num_strings:
                return
            yield "".join(item)
            counter += 1

In [3]:
def _gen_filler(n_cols: int, has_empty: bool = True) -> list[float]:
    fill = []
    for _ in range(n_cols):
        if has_empty:
            fill.append(None)
        # bias towards blanks since most plates are surrounded by blanks
        r = random.randint(1, 6 if has_empty else 3)
        if r == 1:
            fill.append(random.random())
        elif r == 2:
            fill.append(random.randint(1, 1000))
        elif r == 3:
            res = ''.join(random.choices(string.ascii_lowercase + string.ascii_uppercase +
                             string.digits, k=random.randint(5, 50)))
            fill.append(res)
        else:
            fill.append(None)
    return fill
    

In [4]:
def create_nontabular_plate(
    n_rows: int,
    n_cols: int,
    offset: int,
    contents: str = 'Plate',
    n_filler: int = 10
) -> pd.DataFrame:
    cols_before = random.randint(1, n_filler)
    cols_after = random.randint(1, n_filler)
    tot_cols = n_cols + cols_before + cols_after
    rows_before = random.randint(1, n_filler)
    rows_after = random.randint(1, n_filler)
    # Add one for the header
    row_start = rows_before + offset + 1
    results = {'row_start': row_start, 
               'row_end': row_start + (n_rows - 1), # it's inclusive
               'col_start': cols_before, 
               'col_end': cols_before + (n_cols - 1), # it's inclusive
               'contents': contents}
    plate_rows = []
    letters = list(generate_row_letters(n_rows))
    # Generate junk rows before the header
    for row in range(rows_before):
        plate_rows.append(_gen_filler(tot_cols))
    # Generate the header which is a monotonically increasing sequence
    plate_header = (
        _gen_filler(cols_before) + list(range(1, n_cols + 1)) + _gen_filler(cols_after)
    )
    plate_rows.append(plate_header)
    # Fill out the rest of the plate
    for row in range(n_rows):
        data_row = (
            _gen_filler(cols_before - 1)
            + [letters[row]]
            # This is the plate contents
            + _gen_filler(n_cols, has_empty=False)
            + _gen_filler(cols_after)
        )
        plate_rows.append(data_row)
    for row in range(rows_after):
        plate_rows.append(_gen_filler(tot_cols))
    return pd.DataFrame(plate_rows), results

In [5]:
def create_plates(
    n_plates: int,
    n_rows: int,
    n_cols: int,
    return_dict: bool = False,
) -> pd.DataFrame | dict[str, pd.DataFrame]:
    plates: list[pd.DataFrame] = []
    results: list[dict] = []
    offset = 0
    for i in range(n_plates):
        plate_df, result = create_nontabular_plate(n_rows, n_cols, offset)
        offset += plate_df.shape[0]
        plates.append(plate_df)
        results.append(result)
    return {f"plate_{i}": plate_data for i, plate_data in enumerate(plates)} if return_dict else pd.concat(plates, ignore_index=True), results

In [6]:
def _get_plate_from_result_dict(df: pd.DataFrame, result_dict: dict) -> pd.DataFrame:
    row_start, row_end = result_dict['row_start'], result_dict['row_end'] + 1
    col_start, col_end = result_dict['col_start'], result_dict['col_end'] + 1
    proposed_plate = df.iloc[
        row_start:row_end,
        col_start:col_end,
    ]
    return proposed_plate

In [7]:
plates_df, results = create_plates(random.randint(1, 10), 4, 6)
print(results)
_get_plate_from_result_dict(plates_df, results[0])

[{'row_start': 11, 'row_end': 14, 'col_start': 5, 'col_end': 10, 'contents': 'Plate'}, {'row_start': 33, 'row_end': 36, 'col_start': 3, 'col_end': 8, 'contents': 'Plate'}, {'row_start': 47, 'row_end': 50, 'col_start': 10, 'col_end': 15, 'contents': 'Plate'}, {'row_start': 69, 'row_end': 72, 'col_start': 7, 'col_end': 12, 'contents': 'Plate'}, {'row_start': 87, 'row_end': 90, 'col_start': 7, 'col_end': 12, 'contents': 'Plate'}, {'row_start': 99, 'row_end': 102, 'col_start': 5, 'col_end': 10, 'contents': 'Plate'}, {'row_start': 109, 'row_end': 112, 'col_start': 1, 'col_end': 6, 'contents': 'Plate'}, {'row_start': 121, 'row_end': 124, 'col_start': 9, 'col_end': 14, 'contents': 'Plate'}, {'row_start': 143, 'row_end': 146, 'col_start': 4, 'col_end': 9, 'contents': 'Plate'}, {'row_start': 158, 'row_end': 161, 'col_start': 10, 'col_end': 15, 'contents': 'Plate'}]


,5,6,7,8,9,10
11,84peQ41BqMo,None,0.990539,A,0.474304,545.0
12,0.1179,None,gugrWhoYFwTTIxZ6kbE9t,B,7xtoQvcWYgfPvwosYlPjwG,144.0
13,53,None,None,C,914,0.882762
14,None,None,None,D,849,875.0


In [12]:
plates_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51
0,None,477,None,373,None,PQ8cKWElRJFQ1apFTDYycqKeWhCvo3yRXfRUSIFdn76S7,None,None,None,55,NaN,0.443162,NaN,O7hyPDU4Bwuwv,None,None,None,0.39796,None,542,NaN,vTkNRtc8IKHWpOUJRKUSkNS4UTM3J9EwC5L,None,None,None,...,185,NaN,0.83158,None,580,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,rEjrtntwRTVcGJeU3V4jnhEaf3KuufrPxmkn2jKm3,None,None,None,0Z1CWz5X7s0uY,None,None,None,470,NaN,697,NaN,0.234189,None,None,None,None,None,TlsajRUYUfL7,NaN,None,None,Sw4kwebnVvp1duayuPBCzsCJty4o6gAalFn7vl,None,...,None,NaN,709,None,Yjqhz72F3O36qHCQ0Kaa4rhIeOk3AOdBOdghfm0KIkg,None,758.0,None,0.180217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,8CbW14gXRDQ1RLUySGuS6vUqfhAPMgVeHYmds8V07I7qxU,None,389,None,AVwgoXpUmn7i5I,None,346,None,None,NaN,None,NaN,0.212275,None,None,None,None,None,eVZOQ1TUXB7gPdzt5DH0ZyuexwxQW1hUiqy,NaN,209,None,111,None,...,0.046949,NaN,None,None,hiMaIoRriX2lQY2XHgamtc5XtCKsmuqAAW5nas,None,0.607357,None,600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,None,None,None,None,I4JCEnwESW97OSKiq6GZFbKiq9TrlQAQyb8,None,None,None,None,NaN,None,NaN,None,None,None,None,None,None,0.57236,NaN,0.620657,None,BgH23csI54T9t2SLHappWFUfgWDgY0z5bMD,None,...,None,NaN,None,None,RdAea3qg9Cd,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,DgvJ2Oq3gis6IvaNnaN0gwLsedOY5IHOA,None,None,None,None,None,92LzpcA9tZro3YWKrd3STgVfZID,None,533,NaN,None,NaN,0.957788,None,RhloSXqjjj5x4npxhJtM,None,240,None,None,NaN,None,None,128,None,...,874,NaN,441,None,None,None,NaN,None,489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,None,None,None,0.434247,None,None,None,jtLRVC4zIRzV5Ykd0dGcgRH,None,None,NaN,1gX5UaUbASrWECwza,NaN,None,None,34,None,VL6mBm4VVkr,None,Ea7Hk0DhSYCgt18gqDMWb,NaN,0.962792,None,0.730425,None,...,fsWvOdXgB7y3GY0t,NaN,None,None,815,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,None,None,None,None,None,None,None,0.844264,None,7b9pNnjTkKZoGveAMTJ26AdPfvFRJQJ84wtneb0O5uDy,NaN,None,NaN,nIu7LrGOGCIPRw7gDk2sEUB1vssPiUC,None,505,None,561,None,S1YpFWuegqflsUU2Q0JciMFQVg6kHe82j8IwvkQX5UJ9,NaN,701,None,0.805495,None,...,0.596149,NaN,C7c1aBAobp4j,None,None,None,NaN,None,iFcWQNAeddvyzlKNXJ2iY0W7VmPSV6XV33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,None,SynvRpfYOcJqB6r7VMbTjmXW7ydW3tNDr,None,None,None,279,None,None,None,None,NaN,lvL9G,NaN,None,None,eNYyD,None,ODBtsksPfnLgpEwexAHj2rWIY4,None,535,NaN,None,None,xTeVjl3O2vj1BKn4Xwn4xuyMlDRuvqPwojv,None,...,None,NaN,None,None,None,None,NaN,None,0.818626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,None,905,None,None,None,598,None,wCjM6iq3WGNzgYN5DENMKrur5N8BWv3nuO4m23,None,None,NaN,0.699538,NaN,x2SkBBZilNGWVou0fn2y1SUW8Go,None,None,None,369,None,0.438216,NaN,None,None,uKDYBCZT8c2bcXDH,None,...,0.496452,NaN,0.753934,None,wS1t480GaY2,None,NaN,None,623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,None,None,None,VDvB5DrjSsda6mmIjMnG2ZbIDNdY,None,J4MT8,None,0.368324,None,None,NaN,None,NaN,None,None,3,None,682,None,383,NaN,C8cNGe5PhuuKR4coKddEnLToKDL,None,None,None,...,Kqznpgffk2VGyQDKM0uKMdDpZcdOQ9,NaN,0.952975,None,None,None,544.0,None,94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
evals = []
for i in range(100):
    plates_df, results = create_plates(random.randint(1, 10), 4, 6)
    evals.append({"input": plates_df.to_csv(index=False, header=False), "output":results})
for i in range(100):
    plates_df, results = create_plates(random.randint(1, 10), 8, 12)
    evals.append({"input": plates_df.to_csv(index=False, header=False), "output":results})
for i in range(100):
    plates_df, results = create_plates(random.randint(1, 10), 16, 24)
    evals.append({"input": plates_df.to_csv(index=False, header=False), "output":results})
for i in range(10):
    plates_df, results = create_plates(random.randint(1, 10), 32, 48)
    evals.append({"input": plates_df.to_csv(index=False, header=False), "output":results})

In [108]:
with open('gen_evals.json', 'w') as f:
    json.dump(evals, f)

In [114]:
plates_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,None,0lGZsTtZqRq1YGyEHYGFhR0DW7bIfUNSxJlbjCPHQLfa11KwC,None,None,None,None,None,None,None,None,NaN,0.121591,None,966,None,NaN,None,112,None,None,None,0.983377,None,608.0,None,NaN,None,NaN,None,gZO1PtE0nKJ3lwzDGw,None,None,NaN,NaN
1,None,0.2955,None,l5sIozlzJfnOQ67xrmUzpiBVZoB7TD,None,0.269455,None,None,None,None,NaN,129,None,kVtQ5VBE49u,None,NaN,None,None,1,2,3,4,5,6.0,None,NaN,None,NaN,None,None,None,None,NaN,NaN
2,None,722,None,None,None,i6AstG22s8Cb88ktNduWfu2O4ZSMtT18ySfhFlK0d9t,None,None,None,1z3LMVH9UTx5OxbazLSJl8pgxjLLspp7zWZPrJwAf02Nnshd7,NaN,0.029341,None,None,None,NaN,A,0.020699,0.776257,oaqsnXhXqqFyvPybUFaapUftTdEroY303ahWMKxYBIZ,TryYzSOqwPejSMmr4DMJrEhoO55nbsTJ3SY,0.861957,lfBqcQsv2oMWoJCzLygxKIedwyQ7iXT1juIUDJThZdsB3r...,NaN,kmzohBqQcxtPHkJmhODI6Ov2joDNmBMY,NaN,None,NaN,None,None,None,None,NaN,NaN
3,None,0.702402,None,None,None,None,None,YTOxGPEGlSH7KlONI6uSzZyVj7PdJ3eVlf9fFALXmRgUdzcU,None,None,NaN,268,None,0.209087,None,0.4656,B,20us17NGMB5AHUn72EXfiEXupxTMXtyUL5xDUg3MKuYID,70,D28qisjvRiONwW9nKhGuQoHmgHxVppiLJhgg,0.683478,907,0.827364,NaN,None,NaN,None,NaN,None,None,None,None,NaN,NaN
4,None,None,None,None,None,None,None,None,None,None,NaN,None,None,478,None,NaN,C,41,xhHtGvH9bRmofTl6PT4uRMUdZpqCUrHl7nib72o,8,0.013126,0.337339,948,NaN,None,NaN,None,NaN,None,None,None,None,NaN,NaN
5,None,0.15484,None,None,None,None,None,None,None,945,NaN,538,None,KUk8wQH99CzjAR2t94GxwRMwLiXB7WRZhQDk29i7HBxm0i...,None,0.224276,D,340,66,Fv78o91,e32TAsxqeqtjGXrxl9cQc5,NE73bDert9O6LTTieu0pDwOJtYlPjJ78z1A,cHuOBrgAI7jgxAusDWMMUlOx5tRa9KRz4Vem,NaN,None,NaN,None,NaN,None,None,None,None,NaN,NaN
6,None,None,None,None,None,0.531886,None,None,None,0.342786,NaN,None,None,None,None,0.696387,None,DVg1fdLMN3vu,None,None,None,151,None,0.521799,None,308.000000,None,NaN,None,None,None,None,NaN,NaN
7,None,0.045334,None,None,None,None,None,None,None,None,NaN,bYGUODKj0l,None,None,None,546.0,None,None,None,IfBupSHirZ,None,None,None,0.12874,None,0.161239,None,0.758832,None,None,None,k98otDxWQYlNmX7Fr1uVsWi8y,NaN,NaN
8,None,0.439353,None,None,None,None,None,148,None,None,NaN,0.665736,None,0.716956,None,NaN,None,975,None,None,None,158,None,411.0,None,NaN,None,NaN,None,None,None,None,NaN,NaN
9,None,ooMs0vYObIIXBsYwkp,None,0.904097,None,eyODFr,None,None,None,NaN,NaN,NaN,None,None,None,None,None,337,NaN,252,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,None,NaN,None,a6EurjfKbqRwNAsTHOw6LlU6yEoFL26LXPVofBm0
